In [1]:
import scipy
import openmdao.api as om


class ActuatorDisc(om.ExplicitComponent):
    """Simple wind turbine model based on actuator disc theory"""

    def setup(self):

        # Inputs
        self.add_input('a', 0.5, desc="Induced Velocity Factor")
        self.add_input('Area', 10.0, units="m**2", desc="Rotor disc area")
        self.add_input('rho', 1.225, units="kg/m**3", desc="air density")
        self.add_input('Vu', 10.0, units="m/s", desc="Freestream air velocity, upstream of rotor")

        # Outputs
        self.add_output('Vr', 0.0, units="m/s",
                        desc="Air velocity at rotor exit plane")
        self.add_output('Vd', 0.0, units="m/s",
                        desc="Slipstream air velocity, downstream of rotor")
        self.add_output('Ct', 0.0, desc="Thrust Coefficient")
        self.add_output('thrust', 0.0, units="N",
                        desc="Thrust produced by the rotor")
        self.add_output('Cp', 0.0, desc="Power Coefficient")
        self.add_output('power', 0.0, units="W", desc="Power produced by the rotor")
        
        # Every output depends on `a`
        self.declare_partials(of='*', wrt='a', method='cs')

        # Other dependencies
        self.declare_partials(of='Vr', wrt=['Vu'], method='cs')
        self.declare_partials(of=['thrust', 'power'], wrt=['Area', 'rho', 'Vu'], method='cs')

    def compute(self, inputs, outputs):
        """ Considering the entire rotor as a single disc that extracts
        velocity uniformly from the incoming flow and converts it to
        power."""

        a = inputs['a']
        Vu = inputs['Vu']

        qA = .5 * inputs['rho'] * inputs['Area'] * Vu ** 2

        outputs['Vd'] = Vd = Vu * (1 - 2 * a)
        outputs['Vr'] = .5 * (Vu + Vd)

        outputs['Ct'] = Ct = 4 * a * (1 - a)
        outputs['thrust'] = Ct * qA

        outputs['Cp'] = Cp = Ct * (1 - a)
        outputs['power'] = Cp * qA * Vu

In [2]:
prob = om.Problem()
prob.model.add_subsystem('a_disk', ActuatorDisc(),
                         promotes_inputs=['a', 'Area', 'rho', 'Vu']);

In [3]:
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'

prob.model.add_design_var('a', lower=0., upper=1.)

# negative one so we maximize the objective
prob.model.add_objective('a_disk.Cp', scaler=-1)

In [4]:
prob.setup()

prob.set_val('a', .5)
prob.set_val('Area', 10.0, units='m**2')
prob.set_val('rho', 1.225, units='kg/m**3')
prob.set_val('Vu', 10.0, units='m/s')

In [5]:
fail = prob.run_driver()

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.592592590665925
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
Optimization Complete
-----------------------------------


In [6]:
prob.model.list_inputs(val=True, units=True)

4 Input(s) in 'model'

varname  val           units  
-------  ------------  -------
a_disk
  a      [0.33335528]  None   
  Area   [10.]         m**2   
  rho    [1.225]       kg/m**3
  Vu     [10.]         m/s    




[('a_disk.a', {'units': None, 'val': array([0.33335528])}),
 ('a_disk.Area', {'units': 'm**2', 'val': array([10.])}),
 ('a_disk.rho', {'units': 'kg/m**3', 'val': array([1.225])}),
 ('a_disk.Vu', {'units': 'm/s', 'val': array([10.])})]

In [7]:
prob.model.list_outputs(val=True, units=True)

6 Explicit Output(s) in 'model'

varname   val              units
--------  ---------------  -----
a_disk
  Vr      [6.6664472]      m/s  
  Vd      [3.33289439]     m/s  
  Ct      [0.88891815]     None 
  thrust  [544.46236677]   N    
  Cp      [0.59259259]     None 
  power   [3629.62961783]  W    


0 Implicit Output(s) in 'model'




[('a_disk.Vr', {'units': 'm/s', 'val': array([6.6664472])}),
 ('a_disk.Vd', {'units': 'm/s', 'val': array([3.33289439])}),
 ('a_disk.Ct', {'units': None, 'val': array([0.88891815])}),
 ('a_disk.thrust', {'units': 'N', 'val': array([544.46236677])}),
 ('a_disk.Cp', {'units': None, 'val': array([0.59259259])}),
 ('a_disk.power', {'units': 'W', 'val': array([3629.62961783])})]

In [8]:
# Verify the correct outputs

# minimum value
print(prob.get_val('a_disk.Cp'))
print(prob.get_val('a'))

[0.59259259]
[0.33335528]


In [2]:
! openmdao n2 OpenMDAO-examples-wind-actuator-disc.py

In [8]:
! openmdao n2 test2.py

Traceback (most recent call last):
  File "/Users/h1he2li3/opt/anaconda3/envs/dymos_rangemax/bin/openmdao", line 8, in <module>
    sys.exit(openmdao_cmd())
             ^^^^^^^^^^^^^^
  File "/Users/h1he2li3/opt/anaconda3/envs/dymos_rangemax/lib/python3.11/site-packages/openmdao/utils/om.py", line 551, in openmdao_cmd
    options.executor(options, user_args)
  File "/Users/h1he2li3/opt/anaconda3/envs/dymos_rangemax/lib/python3.11/site-packages/openmdao/visualization/n2_viewer/n2_viewer.py", line 738, in _n2_cmd
    _load_and_exec(options.file[0], user_args)
  File "/Users/h1he2li3/opt/anaconda3/envs/dymos_rangemax/lib/python3.11/site-packages/openmdao/utils/file_utils.py", line 196, in _load_and_exec
    exec(code, globals_dict)  # nosec: private, internal use only
    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "test2.py", line 226, in <module>
    prob.setup()
  File "/Users/h1he2li3/opt/anaconda3/envs/dymos_rangemax/lib/python3.11/site-packages/openmdao/core/problem.py", line 958, in setup
   